In [3]:
import os
from mip import *

class DietSolver():

    # Initialize class object
    # input_data has the information about the dishes
    def __init__(self, input_data):
        self.dishes = input_data["dishes"]
        self.costs = input_data["costs"]
        self.components = input_data["components"]
        self.components_ingerdients_amount = input_data["components_ingerdients_amount"]
        self.components_min_amount = input_data["components_min_amount"]
        self.main_dishes = input_data["main_dishes"]
        self.cultural_dishes = input_data["cultural_dishes"]
        self.main_dish_min_rep = input_data["main_dish_min_rep"]
        self.cultural_dish_min_rep = input_data["cultural_dish_min_rep"]
        self.cultural_dish_max_rep = input_data["cultural_dish_max_rep"]
        self.others_max_rep = input_data["others_max_rep"]
        self.total_to_make = input_data["total_to_make"]

        # Create a Python-MIP model
        self.create_model()
        # Create a a variable for each dish
        self.create_variables()
        # Formulate the objective function
        self.create_objective_function()
        # Formulate each constraint
        self.create_constraints()
    

    # Create a Python-MIP model
    def create_model(self):
        self.model = Model(solver_name="GRB", sense=MINIMIZE)
    

    # Create a a variable for each dish
    def create_variables(self):
        self.variables = [
            self.model.add_var(
                name="x_" + dish, 
                lb=0, 
                ub = self.total_to_make, 
                var_type=CONTINUOUS
            )
            for dish in self.dishes
        ]
    

    # Formulate the objective function
    def create_objective_function(self):
        self.model.objective = xsum([
            self.costs[i] * self.variables[i]
            for i in range(len(self.costs))
        ])


    # Formulate each constraint
    def create_constraints(self):
        # Ensures that the minimum amount of each component is match
        components_min_amount = [
            self.model.add_constr(
                xsum([
                    self.components_ingerdients_amount[i][j] 
                    * 
                    self.variables[j]
                    for j in range(len(self.dishes))
                ])
                >= 
                self.components_min_amount[i],

                name="constr_min_amount_" + self.components[i]
            )
            for i in range(len(self.components))
        ]

        # Ensure that the minumum amount of main dishes is match
        main_dish_rep_constraints = [
            self.model.add_constr(
                xsum([
                    self.model.var_by_name("x_" + dish)
                    for dish in self.main_dishes
                ])
                >=
                self.main_dish_min_rep,
                name="constr_min_main_dish"
            )
        ]

        # Ensures that the cultural minimum dishes are match
        cultural_dish_min_rep_constraints = [
            self.model.add_constr(
                self.model.var_by_name("x_" + dish)
                >=
                self.cultural_dish_min_rep,
                name="constr_min_cultural_dish_" + dish
            )
            for dish in self.cultural_dishes
        ]

        # Create a list without main and cultural dishes and 
        other_dishes = list(
            set(self.dishes) 
            - 
            set(self.main_dishes) 
            - 
            set(self.cultural_dishes)
        )

        # For each dish that is not a main dish or a cultural dish
        # Ensures that the maximum number of dishes is not exceeded
        other_dishes_max_rep = [
            self.model.add_constr(
                self.model.var_by_name("x_" + dish)
                <=
                self.others_max_rep,
                name="constr_max_other_dish"
            )
            for dish in other_dishes
        ]

        # Store the constraints
        self.constraints = (
            components_min_amount 
            + 
            main_dish_rep_constraints
            +
            cultural_dish_min_rep_constraints
            +
            other_dishes_max_rep
        )


    def optimize(self):
        self.status = self.model.optimize()

    def print_model(self):
        self.model.write("temp.lp")
        with open("temp.lp", "r") as f_model:
            text = f_model.read()
            print(text)
            os.remove("temp.lp")

    def print_solution(self):
        if (self.status == OptimizationStatus.OPTIMAL):
            print("Solução ótima encontrada")
            print("Custo: " + str(self.model.objective_value))
            for i, var in enumerate(self.variables):
                print(
                    "Quantidade da refeição \"" 
                    + 
                    self.dishes[i] 
                    + 
                    "\": " 
                    + 
                    str(var.x)
                )
            return
        if(self.status == OptimizationStatus.INFEASIBLE):
            print("Modelo infactível")
            return
        
        if (self.status == OptimizationStatus.UNBOUNDED):
            print("Solução ilimitada (f.o. tende ao infinito)")
            return






In [4]:

dishes = [
    "Almôndegas",  
    "Strogonoff", 
    "Churrasco", 
    "Arroz", 
    "Feijão", 
    "Macarrão", 
    "Lasanha",
    "Purê de Batata",
    "Sopa de Legumes",
    "Lentilhas",
    "Ervilhas",
    "Doce de Leite",
    "Doce de Abóbora",
    "Banana",
    "Maçã",
    "Leite"
]

costs_per_person = [
    0.80, 
    0.75, 
    1.10,
    0.05,
    0.15, 
    0.20, 
    0.40, 
    0.20, 
    0.25, 
    0.15,
    0.15,
    0.5,
    0.3,
    0.1,
    0.15,
    0.40
]

components = [
    "proteina",
    "calcio",
    "ferro",
    "calorias"
]


components_ingerdients_amount = [
    [10, 17, 26, 3, 8, 5, 8, 3, 1, 1, 5, 7, 1, 2, 0, 3],
    [4, 4, 5, 0, 0, 0, 4, 0, 0, 0, 0, 250, 13, 8, 6, 123],
    [2, 2, 3, 0, 5, 0, 0, 0, 0, 2, 2, 0, 1, 1, 0, 0],
    [190, 150, 230, 130, 350, 160, 130, 80, 40, 170, 80, 310, 200, 120, 50, 60]
]

components_min_amount = [
    460,
    2000,
    80,
    25000
]


main_dishes = [
    "Almôndegas",  
    "Strogonoff", 
    "Churrasco"
]

cultural_dishes = [
    "Arroz",
    "Feijão"
]

main_dish_min_rep = 20
cultural_dish_min_rep = 10
cultural_dish_max_rep = 20
others_max_rep = 5
total_to_make = 20


In [5]:
input_data = {
    "dishes" : dishes,
    "costs" : costs_per_person,
    "components" : components,
    "components_ingerdients_amount" : components_ingerdients_amount,
    "components_min_amount" : components_min_amount,
    "main_dishes" : main_dishes,
    "cultural_dishes" : cultural_dishes,
    "main_dish_min_rep" : main_dish_min_rep,
    "cultural_dish_min_rep" : cultural_dish_min_rep,
    "cultural_dish_max_rep" : cultural_dish_max_rep,
    "others_max_rep" : others_max_rep,
    "total_to_make" : total_to_make
}
model = DietSolver(input_data=input_data)
model.print_model()
model.optimize()
model.print_solution()

Set parameter Username
Academic license - for non-commercial use only - expires 2025-03-13
Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored
\ LP format - for model browsing. Use MPS format to capture full model detail.
Minimize
  0.8 x_Almôndegas + 0.75 x_Strogonoff + 1.1 x_Churrasco + 0.05 x_Arroz
   + 0.15 x_Feijão + 0.2 x_Macarrão + 0.4 x_Lasanha
   + 0.2 x_Purê_de_Batata + 0.25 x_Sopa_de_Legumes + 0.15 x_Lentilhas
   + 0.15 x_Ervilhas + 0.5 x_Doce_de_Leite + 0.3 x_Doce_de_Abóbora
   + 0.1 x_Banana + 0.15 x_Maçã + 0.4 x_Leite
Subject To
 constr_min_amount_proteina: 10 x_Almôndegas + 17 x_Strogonoff
   + 26 x_Churrasco + 3 x_Arroz + 8 x_Feijão + 5 x_Macarrão + 8 x_Lasanha
   + 3 x_Purê_de_Batata + x_Sopa_de_Legumes + x_Lentilhas + 5 x_Ervilhas
   + 7 x_Doce_de_Leite + x_Doce_de_Abóbora + 2 x_Banana + 3 x_Leite
   >= 460
 constr_min_amount_calcio: 4 x_Almôndegas + 4 x_Strogonoff + 5 x_Churrasco
   + 4 x_Lasanha + 250 x_Doce_de_Leite + 13 x_Doce_de_Abóbora 